In [ ]:
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 7.8 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade nibabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 10.7 MB/s eta 0:00:00
  Attempting uninstall: nibabel
    Found existing installation: nibabel 4.0.2
    Uninstalling nibabel-4.0.2:
      Successfully uninstalled nibabel-4.0.2


In [ ]:
import glob
import os
import SimpleITK as sitk
import nibabel as nib
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import gzip
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.checkpoint import checkpoint
from torchvision.transforms import Resize
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import Lambda, Compose
from scipy.ndimage import zoom
import psutil

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


#Preprocessing and saving the original files as nii.gz

In [ ]:
# # Directory containing the .mhd files
# input_dir = '/content/drive/MyDrive/seg_lung_luna'
# output_dir = '/content/drive/MyDrive/Dataset'

# # Reference size and spacing for all images
# reference_size = [256, 256, 256]  # Example reference size
# reference_spacing = [1.0, 1.0, 1.0]  # Example reference spacing

# # List all .mhd files in the directory
# mhd_files = glob.glob(os.path.join(input_dir, '*.mhd'))

# def process_mhd_file(mhd_file):
#     try:
#         # Read the .mhd file
#         img = sitk.ReadImage(mhd_file)
#         # print(f"Processing {mhd_file}")

#         # Configure the resampler to use the new transformation
#         resampler = sitk.ResampleImageFilter()
#         resampler.SetSize(reference_size)
#         resampler.SetOutputSpacing(reference_spacing)
#         resampler.SetOutputOrigin(img.GetOrigin())
#         resampler.SetOutputDirection(img.GetDirection())
#         resampler.SetInterpolator(sitk.sitkLinear)

#         # Execute the resampling
#         resampled_img = resampler.Execute(img)

#         # Convert the resampled image to a NumPy array
#         volume = sitk.GetArrayFromImage(resampled_img)

#         # Normalize the volume based on the given HU values
#         min_hu = -1000
#         max_hu = 400
#         volume_normalized = (volume - min_hu) / (max_hu - min_hu)

#         # Prepare the affine matrix
#         affine_matrix = np.eye(4)  # Identity matrix for 4x4

#         # Save the preprocessed volume as a compressed .nii.gz file
#         output_filename = os.path.splitext(os.path.basename(mhd_file))[0] + '.nii.gz'
#         output_path = os.path.join(output_dir, output_filename)

#         nii_img = nib.Nifti1Image(volume_normalized, affine_matrix)
#         nib.save(nii_img, output_path)  # Specify compress=True for gzip compression
#         with open(output_path, 'rb') as f_in:
#           with gzip.open(output_path + '.gz', 'wb') as f_out:
#               f_out.writelines(f_in)

#         print(f"Processed {mhd_file} and saved as {output_path}")


#     except Exception as e:
#         print(f"Error processing {mhd_file}: {e}")

# # Use ThreadPoolExecutor for parallel processing
# with ThreadPoolExecutor(max_workers=4) as executor:
#     executor.map(process_mhd_file, mhd_files)


#Creating the custom dataset


In [ ]:
class CTMaskedDataset(Dataset):
    def __init__(self, root_dir, transform=None, target_size=(128, 128)):
        self.root_dir = root_dir
        self.target_size = target_size
        self.scan_folders = [f for f in os.listdir(self.root_dir) if f.endswith('_extracted.nii.gz') or f.endswith('_masked_extracted.nii.gz')]
        print(f"Found {len(self.scan_folders)} NIFTI files in {self.root_dir}")
        self.transform = transform
        self.max_slices = self.calculate_max_slices()

    def calculate_max_slices(self):
        num_slices_per_scan = []
        all_files = os.listdir(self.root_dir)
        nii_files = [f for f in all_files if f.endswith('.nii.gz')]
        for nii_file in nii_files:
            num_slices = len(self.load_nifti(os.path.join(self.root_dir, nii_file)))
            num_slices_per_scan.append(num_slices)
        return max(num_slices_per_scan)

    def __len__(self):
        valid_count = 0
        for nifti_file in self.scan_folders:
            if '_masked_extracted.nii.gz' in nifti_file:
                valid_count += 1
        print("Valid count:", valid_count)
        return valid_count

    def __getitem__(self, idx):
        scan_path = self.root_dir
        all_files = os.listdir(scan_path)
        nii_files = [f for f in all_files if f.endswith('_extracted.nii.gz')]
        masked_files = [f for f in all_files if f.endswith('_masked_extracted.nii.gz')]
        file_pairs = list(zip(nii_files, masked_files))
        original_file, masked_file = file_pairs[idx]
        original_file_path = os.path.join(scan_path, original_file)
        masked_file_path = os.path.join(scan_path, masked_file)
        if not os.path.isfile(original_file_path) or not os.path.isfile(masked_file_path):
            print(f"Missing file for {original_file}, returning zeros for missing data.")
            image = np.zeros_like(np.random.rand(1, 128, 128))
            mask = np.zeros_like(np.random.rand(1, 128, 128))
        else:
            image = torch.from_numpy(self.load_nifti(original_file_path)).float()
            mask = torch.from_numpy(self.load_nifti(masked_file_path)).float()

        padding_needed = self.target_size[1] - mask.shape[1]
        padding_needed = max(0, padding_needed)
        padded_image = F.pad(image, (0, padding_needed, 0, 0), "constant", 0)
        padded_mask = F.pad(mask, (0, padding_needed, 0, 0), "constant", 0)
        padded_mask = self.normalize_mask(padded_mask)

        if self.transform:
            padded_image = self.transform(padded_image)
            padded_mask = self.transform(padded_mask)
        return image, mask

    def load_nifti(self, nifti_file):
        nifti_data = nib.load(nifti_file)
        nifti_array = np.array(nifti_data.get_fdata())
        return nifti_array

    def normalize_mask(self, mask_array):
        mask_min = mask_array.min()
        mask_max = mask_array.max()
        mask_array = (mask_array - mask_min) / (mask_max - mask_min)
        mask_array = mask_array.type(torch.uint8)
        return mask_array

def custom_collate_fn(batch):
    images = []
    masks = []
    for item in batch:
        image, mask = item
        # Resize both image and mask to a fixed size
        resize = Resize((128, 128))
        image = resize(image.unsqueeze(0)).squeeze(0)
        mask = resize(mask.unsqueeze(0)).squeeze(0)

        # Check if the mask needs padding to match the target size
        padding_needed = 128 - mask.shape[1]
        padding_needed = max(0, padding_needed)
        mask_padded = F.pad(mask, (0, padding_needed, 0, 0), "constant", 0)

        images.append(image)
        masks.append(mask_padded)

    # Stack images and masks
    images = torch.stack(images, dim=0)
    masks = torch.stack(masks, dim=0)

    return images, masks




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Res2Net definition

In [ ]:
class Res2NetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, scale=4):
        super(Res2NetBlock, self).__init__()
        self.scale = scale

        # 1x1x1 conv to reduce input channels to scale * out_channels
        reduced_channels = scale * out_channels
        self.conv1x1x1 = nn.Conv3d(in_channels, reduced_channels, kernel_size=1)

        # Grouped convolutions within Res2Net block
        self.conv3x3x3_groups = nn.ModuleList([
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1, groups=scale)
            for _ in range(scale)
        ])

        # 3D Squeeze-and-Excitation (SE) block
        self.se_block = nn.Sequential(
            nn.AdaptiveAvgPool3d(1),
            nn.Conv3d(reduced_channels, reduced_channels // 16, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(reduced_channels // 16, reduced_channels, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Apply 1x1x1 convolution
        x = self.conv1x1x1(x)

        # Split into groups and apply grouped convolutions
        x_splits = torch.chunk(x, self.scale, dim=1)
        out_splits = [conv(split) for conv, split in zip(self.conv3x3x3_groups, x_splits)]
        out = torch.cat(out_splits, dim=1)

        # Apply SE block for channel-wise attention
        se_weights = self.se_block(out)
        out = out * se_weights

        return out




#UNet definition

In [ ]:
import torch
import torch.nn as nn
from torch.utils.checkpoint import checkpoint
import gc

class ModifiedUNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, base_channels=256):
        super(ModifiedUNet, self).__init__()
        self.encoder1 = self.conv_block(in_channels, base_channels)
        self.encoder2 = self.res2net_block(base_channels, base_channels * 2)
        self.encoder3 = self.res2net_block(base_channels * 2, base_channels * 4)
        self.encoder4 = self.conv_block(base_channels * 4, base_channels * 8)


        self.decoder3 = self.conv_block(base_channels * 8, base_channels * 4)
        self.decoder2 = self.res2net_block(base_channels * 4 + base_channels * 4, base_channels * 2)
        self.decoder1 = self.conv_block(base_channels * 2 + base_channels * 2, base_channels)
        self.final_conv = nn.Conv3d(base_channels, out_channels, kernel_size=1)
        print(f"GPU memory allocated after operations: {torch.cuda.memory_allocated()} bytes")


    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def res2net_block(self, in_channels, out_channels):
        return nn.Sequential(
            Res2NetBlock(in_channels, out_channels),  # Assuming Res2NetBlock is defined elsewhere
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Use checkpoint for the entire forward pass
        x = checkpoint(self.encoder1, x)
        x = checkpoint(self.encoder2, x)
        x = checkpoint(self.encoder3, x)
        x = checkpoint(self.encoder4, x)

        x = checkpoint(self.decoder3, x)
        x = torch.cat([x, checkpoint(self.encoder3, x)], dim=1)  # Concatenate with encoder3 output
        x = checkpoint(self.decoder2, x)
        x = torch.cat([x, checkpoint(self.encoder2, x)], dim=1)  # Concatenate with encoder2 output
        x = checkpoint(self.decoder1, x)

        out = self.final_conv(x)
        return out


Training loop

In [ ]:
import os
import numpy as np
import torch
from torch.nn import functional as F
import nibabel as nib
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import Lambda, Compose, ToTensor
from torchvision.transforms.functional import to_tensor
from torch.optim import Adam
import psutil
from torch.cuda.amp import autocast, GradScaler
import gc

# Set device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def resize_np(arr, size):
    scale_factor = min(size[0] / arr.shape[0], size[1] / arr.shape[1])

    # Initialize an empty array to hold the resized image
    resized_arr = np.empty((size[0], size[1], arr.shape[-1]))

    # Resize each channel individually
    for ch in range(arr.shape[-1]):
        resized_arr[:, :, ch] = zoom(arr[:, :, ch], (scale_factor, scale_factor), order=1)

    return resized_arr

# Define your dataset path
root_dir = '/content/drive/MyDrive/ct_scan'

# Initialize your dataset
dataset = CTMaskedDataset(root_dir=root_dir, transform=Compose([Lambda(lambda x: resize_np(x, (128, 128))), ToTensor()]))

# Split the dataset into training, testing, and validation sets
train_size = int(0.7 * len(dataset))
test_size = int(0.15 * len(dataset))
val_size = len(dataset) - train_size - test_size
train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

# Create separate DataLoaders for each phase
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=custom_collate_fn, pin_memory=True)
print(f"Number of workers: {train_loader.num_workers}")
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn, pin_memory=True)

# Initialize Model, Loss Function, and Optimizer
model = ModifiedUNet(in_channels=1, out_channels=1).to(device)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

num_epochs = 10  # Define the number of epochs

print(f"Initial GPU memory allocated: {torch.cuda.memory_allocated()} bytes")
# Training loop
scaler = GradScaler()
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for batch_idx, (images, masks) in enumerate(train_loader):
        images, masks = images.to(device), masks.to(device)
        images = images.unsqueeze(0)  # This line seems unnecessary unless you're intentionally adding an extra dimension for some reason
        optimizer.zero_grad()
        # print(torch.cuda.memory_summary(device=None, abbreviated=False))

        # Use autocast to enable mixed precision
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, masks)

        # Scale the loss and call backward
        scaler.scale(loss).backward()

        # Unscales the gradients and calls optimizer.step
        scaler.step(optimizer)

        # Updates the scale for next iteration
        scaler.update()

        running_loss += loss.item()
        if True:  # Always true, so this block runs for every batch
          print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {running_loss/(batch_idx+1):.4f}")
          running_loss = 0.0
          gc.collect()


    # Evaluate on test and validation sets
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        test_loss = 0.0
        val_loss = 0.0
        for images, masks in test_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            test_loss += criterion(outputs, masks).item()
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, masks).item()

    print(f"Test Loss: {test_loss/len(test_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}")

    # Save the model after each epoch
    model_name = f"modified_unet_epoch_{epoch + 1}.pt"
    model_path = os.path.join('saved_models', model_name)
    torch.save(model.state_dict(), model_path)
    print(f"Model saved at {model_path}")

# Save the final model
final_model_path = os.path.join('saved_models', "modified_unet_final.pt")
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved at {final_model_path}")

Found 134 NIFTI files in /content/drive/MyDrive/ct_scan
Valid count: 67
Valid count: 67
Valid count: 67
Valid count: 67
Number of workers: 0
GPU memory allocated after operations: 0 bytes
Initial GPU memory allocated: 0 bytes


/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
